In [1]:
#this implementation has been derived from: https://github.com/hannureittu/Community-panning/blob/master/AdevelopeKoe.py


import time
from pyqubo import Array
import numpy as np
import matplotlib.pyplot as plt
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod.reference.samplers import ExactSolver
from dimod import BinaryQuadraticModel
from dimod import Vartype
from dwave.system.composites import FixedEmbeddingComposite
import networkx as nx
import dwave_networkx as dnx
import dimod
from dwave.system.composites import VirtualGraphComposite
from minorminer import find_embedding
from random import random
from dimod.reference.samplers import SimulatedAnnealingSampler
from numpy import genfromtxt
from dwave_qbsolv import QBSolv
import neal
import itertools
import random
import array as arr
from matplotlib import pyplot as plt

In [11]:
def modul_by_regularity(G, method='sim_anneal'):
    k=int(np.sqrt(len(G.nodes())))
    a=nx.to_numpy_array(G)
    run_time=[]
    communities=[]
    #a = genfromtxt('a250.csv', delimiter=',') #read adjacency matrix

    #plt.matshow(a) (print a)
    #plt.show()
    n=len(G)
    li=list(range(1,n+1))
    # Create qubo from the graph matrix            

    ra=np.random.randint(2, size=n) #random partitioning of nodes in two classes lis1 and lis2
    lis1=[]
    lis2=[]
    for i in range(n):
        if ra[i]==0:
            lis1.append(i)
        else:
            lis2.append(i)
    #nodes of graph with adjacency matrix are divided uniformly at random in two sets, described
    #by lists list1 and list2
    coss=[] #values of energy per node at each round
    liis1=lis1
    liis2=lis2
    
    
    for je in range(k): #tryes to find communities in k rounds
        print("Round"+str(je+1))
        energypernode=[]
        stop=False
        if je>0: #removes found communities in each round
            
            for i in range(len(so1)):
                if so1[i] not in liis1:
                    stop=True      
            for i in range(len(so2)):
                if so2[i] not in liis2:
                    stop=True
            if stop:
                print("Stopped: Last found solution not in liis anymore") #stops in case there is no improvement in first iteration of inner loop 
                #--> no update of solu --> solu nodes not in liis as they have been removed in last round
                break
            for i in range(len(so1)):
                liis1.remove(so1[i])
            for i in range(len(so2)):
                liis2.remove(so2[i])
        
        lis1=liis1
        lis2=liis2    
        n1=len(lis1)
        n2=len(lis2)
        if n1==0 or n2==0: #stops if all communities have been detected
            print("No nodes in one of the lists anymore")
            break
        print("verbleibende Knoten:"+str(n1+n2)+str(lis1)+str(lis2))

        b = [[0 for x in range(n2)] for y in range(n1)] #update of adjacency matrix
        for i in range(n1):
            for j in range(n2):
                b[i][j]=a[lis1[i]][lis2[j]]
        #plt.matshow(b)
        #plt.show() #print b-matrix by taking off #'s
        #b is the block of adjacency matrix of bipartite graph with bipartition list1 and list2


        b=(np.asarray(b))
    
        db=np.sum(b) 
        db=db/(n1*n2) #db is the link density of the bipartite graph
        db1=db #db1 is density at the beginning
        ded=[db1]
        emax=5 #emax is number of max iterations at each round
        eni=0  #energy of previous  step
       
        for e in range(emax):        
            
            n1=len(lis1)
            n2=len(lis2)

            b = [[0 for x in range(n2)] for y in range(n1)] 
            for i in range(n1):
                for j in range(n2):
                    b[i][j]=a[lis1[i]][lis2[j]]
            #plt.matshow(b)
            #plt.show() print b-matrix by taking off #'s
            #b is the block of adjacency matrix of bipartite graph with bipartition list1 and list2
            b=(np.asarray(b))
                
            db=np.sum(b)
            db=db/(n1*n2)
            #db is link density of bipartite graph
            ded.append(db)
           # print("Ded" + str(ded))
            db1=db
            #plt.matshow(b)
           # plt.show()



            M=db-b # qubo kernel
        
            # Create qubo:
            #n1 = b.shape[0]
            #n2 = b.shape[1]
            qubo={}
            nn=n1+n2
            s = ['']*nn
            for i in range(nn):
                s[i]="s_"+str(i)
            
            for i in range(n1):
                for j in range(n2):
                    qubo[(s[i],s[j+n1])]=M[i][j]

            la=lis1+lis2 #la has id's of nodes in a bipartite graph b
            
            # Create BinaryQuadraticModel
            bqm = BinaryQuadraticModel({}, qubo, 0, Vartype.BINARY)
            #print(bqm)
        

            if method=='sim_anneal': #methods are sim_anneal (for simulated annealing) or dwave_sampler (for quantum annealing)
                en=0 #energy of best solution       
                for gg in range(1):
                    st = time.time()
                    sampler = neal.SimulatedAnnealingSampler()
                    
                    
                    response = sampler.sample(bqm)
                    #print(response)
                    energ_classic=[]
                    conf_classic=[]
                    et = time.time()
                    ex_time=(et-st)*1000 #runtime in milliseconds
                    run_time.append(ex_time)

                    #print("Simulated annealing, elapsed time: " + str(endTime - startTime))

                    #print("\nResults from simulated annealing sampler:")
                    i = 0
                    ss=0
                    d2=[]

                    for datum in response.data(['sample', 'energy']):
                        energ_classic.append(datum.energy) #list of energy values    
                        if en>datum.energy:
                            en=datum.energy
                            ss=i
                        con=[]
                        for h in range(nn):
                            con.append(datum.sample[s[h]])
                            
                    #print("Sample, Energy", datum.sample, datum.energy)
                        conf_classic.append(con)
                        d2=np.array(conf_classic[ss])

                    if(i >= 10):
                        break
                    i += 1
                        
            elif method=='dwave_sampler':
                en=0 #energy of best solution
                sampler=EmbeddingComposite(DWaveSampler()) #using automatical minor embedding
                response = sampler.sample(bqm)
                energ_classic=[]
                conf_classic=[]
                
                timing=response.info["timing"]
                ex_time=(timing['qpu_access_time']+timing['post_processing_overhead_time'])*0.001 #runtime in milliseconds
                run_time.append(ex_time)
                
                i = 0
                ss=0

                for datum in response.data(['sample', 'energy']):
                    energ_classic.append(datum.energy) #list of energy values    
                    if en>datum.energy:
                        en=datum.energy
                        ss=i
                    con=[]
                    for h in range(nn):
                        con.append(datum.sample[s[h]])
                            
                    #print("Sample, Energy", datum.sample, datum.energy)
                    conf_classic.append(con)
                    
                    d2=np.array(conf_classic[ss])

                    if(i >= 10):
                        break
                    i += 1
                
                
            lis11=[]
        
            for i in range(n1):
                 if(d2[i]==1):#if the spin has value 1, corresponding node is added
                    lis11.append(la[i])
            lis22=[]
            for i in range(n1,n1+n2):
                if(d2[i]==1):
                    lis22.append(la[i])
            lis1=lis11
            lis2=lis22                
            n1=len(lis1)
            n2=len(lis2)
            if n1==0 or n2==0:
                break
            ep=en/(n1*n2)          
            
            if eni>ep: #update the solution of this round and its energy value
                eni=ep
                solu=lis1+lis2
                so1=np.array(lis1)
                so2=np.array(lis2)
            if ep>eni: #stopping criteria in each round
                coss.append(eni)
                break
          
            energypernode.append(en/n1/n2)
            #print(energypernode)
        
            print(solu) # solu is list of nodes in current community 
           
            #print(np.array(run_time).sum())
        communities.append(set(solu))
           
    rest=[] #classify rest as a single community in case of first stopping criteria mentioned above (no initial improvement)
    for i in (set(G.nodes)-(set().union(*communities))):
        rest.append(i)
    rest=set(rest)
    communities.append(rest)
    communities=np.unique(np.array(communities))
    
    run_time=np.array(run_time)
    total_time=(run_time.sum())
    
    print("End")
    #print(coss)
    
    return (communities, total_time) #time in milliseconds